# Srock Strategy - Factor Strategy 0002 - Factor-Based Stock Selection

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [2]:
sd = '2015-12-31'
ed = '2026-02-13'

In [3]:
sql_factor =  f"""
SELECT
    date,
    instrument,
    ps_ttm AS factor
FROM cn_stock_prefactors
WHERE sw2021_level1 IN ('110000', '210000', '220000', '230000', '240000', '270000', '280000', '310000', '320000', '330000', '340000', '350000', '360000', '370000', '410000', '420000', '430000', '440000', '450000', '460000', '470000', '480000', '490000', '510000', '630000', '650000', '710000', '720000', '730000', '740000', '760000', '770000')
AND instrument NOT LIKE '%BJ%'
QUALIFY c_group_pct_rank(sw2021_level2, ps_ttm) < 0.20 
AND c_group_pct_rank(sw2021_level2, total_operating_revenue_yoy_lf) > 0.67
AND net_profit_yoy_lf > 0
"""

In [4]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_factor}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 10
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        float_market_cap / c_sum(float_market_cap) AS position, 
    FROM data_filter JOIN cn_stock_valuation USING (date, instrument) JOIN mldt_cn_stock_calendar_daily USING (date)
    WHERE is_month_end_trade = 1
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [5]:
df_strategy = dai.query(sql_trade, filters = {"date":[sd, ed]}).df()
df_strategy

,date,instrument,score,score_rank,position
0,2015-12-31,000626.SZ,0.170867,1.0,0.055929
1,2015-12-31,600057.SH,0.256267,2.0,0.046145
2,2015-12-31,002210.SZ,0.402541,3.0,0.080966
3,2015-12-31,600973.SH,0.477680,4.0,0.040996
4,2015-12-31,600686.SH,0.488058,5.0,0.054318
...,...,...,...,...,...
1215,2026-01-30,000736.SZ,0.209529,6.0,0.026210
1216,2026-01-30,601607.SH,0.228557,7.0,0.296986
1217,2026-01-30,600697.SH,0.316284,8.0,0.013120
1218,2026-01-30,001287.SZ,0.327141,9.0,0.071016


## 3. Backtest Strategy

In [6]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    target_instruments  = list(df_now["instrument"])
    holding_instruments = list(context.get_account_positions().keys())

    for instrument in holding_instruments:
        if instrument not in target_instruments:
            context.order_target_percent(instrument, 0)
        
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 19:14:06] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 19:14:06] [info     ] 2015-12-31, 2026-01-30, , , instruments=122
[2026-02-18 19:14:07] [info     ] bigtrader module V2.2.0
[2026-02-18 19:14:07] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
order_target_percent: price is null for '000626.SZ' at '2015-12-31 15:00:00'
order_target_percent: price is null for '000626.SZ' at '2016-01-29 15:00:00'
order_target_percent: price is null for '300226.SZ' at '2016-04-29 15:00:00'
order_target_percent: price is null for '300226.SZ' at '2016-05-31 15:00:00'
order_target_percent: price is null for '600973.SH' at '2016-11-30 15:00:00'
order_target_percent: price is null for '000034.SZ' at '2017-03-31 15:00:00'
order_target_percent: price is null for '600710.SH' at '2017-04-28 15:00:00'
order_target_percent: price is null for '000034.SZ' at '2017-04-28 15:00:00'
order_target_percent: price is null for '600710.SH' at '2017-05-31 15:00:00'
order_target_percent: price is 

[2026-02-18 19:14:14] [info     ] bigtrader.v34 运行完成 [7.549s].
